![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [53]:
library(readr)
library(dplyr)
library(forcats)

## IMPORTING THE DATASET

In [54]:
ds_jobs_clean <- read.csv("customer_train.csv")
head(ds_jobs_clean)

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
,<int>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
1,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,,,1,36,1
2,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
3,11561,city_21,0.624,,No relevant experience,Full time course,Graduate,STEM,5,,,never,83,0
4,33241,city_115,0.789,,No relevant experience,,Graduate,Business Degree,<1,,Pvt Ltd,never,52,1
5,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0
6,21651,city_176,0.764,,Has relevant experience,Part time course,Graduate,STEM,11,,,1,24,1


In [55]:
glimpse(ds_jobs_clean)

Rows: 19,158
Columns: 14
$ student_id             <int> 8949, 29725, 11561, 33241, 666, 21651, 28806, 4…
$ city                   <chr> "city_103", "city_40", "city_21", "city_115", "…
$ city_development_index <dbl> 0.920, 0.776, 0.624, 0.789, 0.767, 0.764, 0.920…
$ gender                 <chr> "Male", "Male", "", "", "Male", "", "Male", "Ma…
$ relevant_experience    <chr> "Has relevant experience", "No relevant experie…
$ enrolled_university    <chr> "no_enrollment", "no_enrollment", "Full time co…
$ education_level        <chr> "Graduate", "Graduate", "Graduate", "Graduate",…
$ major_discipline       <chr> "STEM", "STEM", "STEM", "Business Degree", "STE…
$ experience             <chr> ">20", "15", "5", "<1", ">20", "11", "5", "13",…
$ company_size           <chr> "", "50-99", "", "", "50-99", "", "50-99", "<10…
$ company_type           <chr> "", "Pvt Ltd", "", "Pvt Ltd", "Funded Startup",…
$ last_new_job           <chr> "1", ">4", "never", "never", "4", "1", "1", ">4…
$ training_hour

In [56]:
head(ds_jobs_clean)
tail(ds_jobs_clean)

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
,<int>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
1,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,,,1,36,1
2,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
3,11561,city_21,0.624,,No relevant experience,Full time course,Graduate,STEM,5,,,never,83,0
4,33241,city_115,0.789,,No relevant experience,,Graduate,Business Degree,<1,,Pvt Ltd,never,52,1
5,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0
6,21651,city_176,0.764,,Has relevant experience,Part time course,Graduate,STEM,11,,,1,24,1


,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
,<int>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
19153,29754,city_103,0.920,Female,Has relevant experience,no_enrollment,Graduate,Humanities,7,10-49,Funded Startup,1,25,0
19154,7386,city_173,0.878,Male,No relevant experience,no_enrollment,Graduate,Humanities,14,,,1,42,1
19155,31398,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,14,,,4,52,1
19156,24576,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,4,44,0
19157,5756,city_65,0.802,Male,Has relevant experience,no_enrollment,High School,,<1,500-999,Pvt Ltd,2,97,0
19158,23834,city_67,0.855,,No relevant experience,no_enrollment,Primary School,,2,,,1,127,0


## Changing data type to match format (e.g : Columns containing integer values must be stored as integers (<int>)
	

In [57]:
# Change the data type of relevant_experience to factor
# Change the data type of enrolled_university to factor
# Change the data type of education_level to factor
# Change the data type of major_discipline to factor
# Change the data type of gender to factor


ds_jobs_clean$relevant_experience <- as.factor(ds_jobs_clean$relevant_experience)
ds_jobs_clean$enrolled_university <- as.factor(ds_jobs_clean$enrolled_university)
ds_jobs_clean$education_level <- as.factor(ds_jobs_clean$education_level)
ds_jobs_clean$major_discipline <- as.factor(ds_jobs_clean$major_discipline)
ds_jobs_clean$experience <- as.integer(ds_jobs_clean$experience)


summary(ds_jobs_clean)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


   student_id        city           city_development_index    gender         
 Min.   :    1   Length:19158       Min.   :0.4480         Length:19158      
 1st Qu.: 8554   Class :character   1st Qu.:0.7400         Class :character  
 Median :16982   Mode  :character   Median :0.9030         Mode  :character  
 Mean   :16875                      Mean   :0.8288                           
 3rd Qu.:25170                      3rd Qu.:0.9200                           
 Max.   :33380                      Max.   :0.9490                           
                                                                             
              relevant_experience       enrolled_university
 Has relevant experience:13792                    :  386   
 No relevant experience : 5366    Full time course: 3757   
                                  no_enrollment   :13817   
                                  Part time course: 1198   
                                                           
                

## Collapse the company_size levels 

In [58]:

ds_jobs_clean$company_size <- ifelse(
  ds_jobs_clean$company_size %in% c('<10'),
  'Micro',
  ifelse(
    ds_jobs_clean$company_size %in% c('50-99','10-49'),
    'Small',
    ifelse(
      ds_jobs_clean$company_size %in% c('100-499','500-999'),
      'Medium',
      ifelse(
        ds_jobs_clean$company_size %in% c('5000-9999', '10000+', '1000-4999'),
        'Large',
        'Other'  # or another default category
      )
    )
  )
)


table(ds_jobs_clean$company_size)


 Large Medium  Micro  Other  Small 
  3910   3448   1308   5938   4554 

## Collapse the experience levels :

In [59]:
ds_jobs_clean$experience <- as.character(ds_jobs_clean$experience)
unique(ds_jobs_clean$experience)

library(tidyr)

library(tidyr)

# Convert 'experience' column to numeric
ds_jobs_clean$experience <- as.numeric(ds_jobs_clean$experience)

# Replace NA with 0 in the 'experience' column
ds_jobs_clean$experience <- replace_na(ds_jobs_clean$experience, 0)


[1] NA   "15" "5"  "11" "13" "7"  "17" "2"  "16" "1"  "4"  "10" "14" "18" "19"
[16] "12" "3"  "6"  "9"  "8"  "20"

In [60]:
library(dplyr)
library(forcats)

ds_jobs_clean$experience <- as.character(ds_jobs_clean$experience)
unique(ds_jobs_clean$experience)

# Assuming ds_jobs_clean is your dataframe
ds_jobs_clean <- ds_jobs_clean %>%
  mutate(experience = fct_collapse(experience,
    '<5' = c('<1', as.character(1:4)),
    '5-10' = as.character(5:10),
    '>10' = c(as.character(11:20), '>20')
  ))

ds_jobs_clean$experience 

[1] "0"  "15" "5"  "11" "13" "7"  "17" "2"  "16" "1"  "4"  "10" "14" "18" "19"
[16] "12" "3"  "6"  "9"  "8"  "20"

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `experience = fct_collapse(...)`.
Caused by warning:
! Unknown levels in `f`: <1, >20”


[1] 0    >10  5-10 0    0    >10  5-10 >10  5-10 >10  <5   5-10 0    <5  
   [15] 5-10 >10  <5   <5   5-10 <5   >10  >10  5-10 0    5-10 <5   >10  5-10
   [29] >10  >10  0    >10  5-10 0    >10  5-10 <5   0    >10  <5   >10  <5  
   [43] 5-10 5-10 5-10 5-10 5-10 >10  >10  >10  <5   5-10 5-10 <5   <5   0   
   [57] <5   0    <5   0    5-10 5-10 >10  0    <5   5-10 <5   >10  >10  <5  
   [71] 0    0    <5   5-10 0    5-10 5-10 <5   5-10 <5   >10  0    0    5-10
   [85] <5   >10  0    <5   >10  5-10 <5   <5   <5   5-10 5-10 5-10 <5   <5  
   [99] 5-10 5-10 5-10 0    >10  5-10 >10  0    5-10 <5   0    0    5-10 5-10
  [113] <5   <5   5-10 0    <5   5-10 5-10 5-10 >10  0    >10  5-10 5-10 >10 
  [127] <5   0    >10  5-10 0    >10  0    >10  <5   0    <5   <5   <5   5-10
  [141] 5-10 5-10 5-10 0    0    0    >10  >10  5-10 <5   5-10 <5   0    >10 
  [155] 0    5-10 >10  >10  5-10 >10  >10  0    0    5-10 >10  >10  0    0   
  [169] 0    0    5-10 0    5-10 <5   <5   0    0    5-10 0    <5   5-10 0   
  [183] 5-10 0    0    <5   5-10 0    >10  >10  >10  <5   >10  <5   <5   >10 
  [197] >10  0    >10  0    5-10 <5   5-10 5-10 <5   5-10 <5   0    0    <5  
  [211] 0    <5   <5   <5   <5   5-10 0    <5   0    5-10 5-10 0    5-10 <5  
  [225] 5-10 <5   0    <5   5-10 >10  5-10 5-10 <5   >10  5-10 >10  <5   >10 
  [239] 0    5-10 5-10 <5   >10  5-10 0    5-10 5-10 <5   5-10 5-10 >10  0   
  [253] 5-10 5-10 <5   <5   <5   >10  5-10 0    0    >10  >10  >10  >10  <5  
  [267] <5   5-10 >10  5-10 0    <5   5-10 0    5-10 5-10 >10  5-10 5-10 <5  
  [281] 5-10 0    5-10 5-10 5-10 5-10 0    5-10 5-10 >10  <5   <5   0    0   
  [295] <5   >10  5-10 5-10 >10  <5   5-10 <5   5-10 <5   5-10 <5   <5   <5  
  [309] 0    <5   0    <5   0    0    >10  <5   <5   0    <5   >10  >10  0   
  [323] >10  >10  >10  >10  >10  5-10 <5   5-10 5-10 <5   5-10 >10  5-10 5-10
  [337] <5   0    >10  >10  <5   >10  <5   <5   5-10 <5   5-10 <5   <5   5-10
  [351] >10  <5   0    >10  5-10 5-10 0    >10  5-10 >10  0    >10  0    >10 
  [365] 5-10 5-10 <5   5-10 5-10 >10  0    0    <5   0    0    0    >10  >10 
  [379] 5-10 <5   >10  <5   5-10 0    >10  >10  0    0    5-10 >10  5-10 0   
  [393] 5-10 >10  >10  0    0    <5   <5   <5   5-10 >10  0    0    >10  5-10
  [407] 5-10 <5   >10  >10  <5   5-10 0    5-10 5-10 <5   0    <5   <5   0   
  [421] 5-10 5-10 <5   0    <5   >10  5-10 5-10 5-10 <5   5-10 0    <5   5-10
  [435] 0    5-10 <5   <5   <5   >10  5-10 <5   >10  <5   5-10 5-10 0    5-10
  [449] 5-10 >10  0    5-10 5-10 <5   5-10 5-10 5-10 5-10 5-10 5-10 0    >10 
  [463] 0    <5   5-10 <5   <5   0    5-10 5-10 >10  5-10 <5   5-10 <5   0   
  [477] <5   5-10 5-10 <5   0    0    5-10 >10  0    5-10 0    0    0    >10 
  [491] 5-10 0    5-10 <5   5-10 0    5-10 >10  >10  5-10 5-10 >10  5-10 5-10
  [505] 5-10 <5   0    5-10 0    <5   <5   5-10 0    <5   >10  0    5-10 >10 
  [519] 0    <5   5-10 0    >10  <5   5-10 >10  <5   5-10 <5   <5   5-10 5-10
  [533] 5-10 >10  >10  <5   5-10 5-10 5-10 >10  0    5-10 5-10 5-10 0    <5  
  [547] 5-10 5-10 5-10 5-10 >10  >10  >10  0    >10  >10  >10  0    0    5-10
  [561] <5   <5   <5   0    >10  <5   5-10 0    >10  <5   5-10 5-10 >10  5-10
  [575] <5   5-10 5-10 >10  5-10 0    0    5-10 0    0    5-10 5-10 >10  0   
  [589] 0    0    <5   <5   >10  <5   0    <5   >10  5-10 5-10 5-10 0    >10 
  [603] >10  >10  <5   <5   5-10 5-10 >10  5-10 5-10 >10  >10  >10  0    5-10
  [617] 0    0    5-10 5-10 5-10 5-10 5-10 <5   >10  <5   0    5-10 >10  >10 
  [631] <5   <5   >10  5-10 <5   5-10 >10  5-10 0    5-10 <5   <5   0    <5  
  [645] 0    5-10 0    0    >10  5-10 5-10 5-10 <5   5-10 <5   5-10 5-10 <5  
  [659] <5   0    >10  5-10 0    5-10 >10  5-10 5-10 <5   0    5-10 5-10 <5  
  [673] <5   >10  5-10 5-10 <5   >10  5-10 >10  5-10 5-10 5-10 0    <5   5-10
  [687] >10  <5   0    0    5-10 5-10 5-10 5-10 0    <5   5-10 >10  5-10 0   
  [701] >10  5-10 5-10 5-10 <5   >10  0    5-10 0    0    5-10 5-10 <5   >10 
  [715] >10  0    >10  5-1

In [61]:
# Check the distribution of major_discipline
table(ds_jobs_clean$major_discipline)

# Check the distribution of experience
table(ds_jobs_clean$experience)

# Check the distribution of company size
table(ds_jobs_clean$company_size)


                           Arts Business Degree      Humanities        No Major 
           2813             253             327             669             223 
          Other            STEM 
            381           14492 


   0   <5 5-10  >10 
3873 4433 6441 4411 


 Large Medium  Micro  Other  Small 
  3910   3448   1308   5938   4554 

In [62]:
# Display the first few rows of the cleaned dataframe
head(ds_jobs_clean)
glimpse(ds_jobs_clean)

					

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
,<int>,<chr>,<dbl>,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>,<chr>,<int>,<int>
1,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,0,Other,,1,36,1
2,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,>10,Small,Pvt Ltd,>4,47,0
3,11561,city_21,0.624,,No relevant experience,Full time course,Graduate,STEM,5-10,Other,,never,83,0
4,33241,city_115,0.789,,No relevant experience,,Graduate,Business Degree,0,Other,Pvt Ltd,never,52,1
5,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,0,Small,Funded Startup,4,8,0
6,21651,city_176,0.764,,Has relevant experience,Part time course,Graduate,STEM,>10,Other,,1,24,1


Rows: 19,158
Columns: 14
$ student_id             <int> 8949, 29725, 11561, 33241, 666, 21651, 28806, 4…
$ city                   <chr> "city_103", "city_40", "city_21", "city_115", "…
$ city_development_index <dbl> 0.920, 0.776, 0.624, 0.789, 0.767, 0.764, 0.920…
$ gender                 <chr> "Male", "Male", "", "", "Male", "", "Male", "Ma…
$ relevant_experience    <fct> Has relevant experience, No relevant experience…
$ enrolled_university    <fct> no_enrollment, no_enrollment, Full time course,…
$ education_level        <fct> Graduate, Graduate, Graduate, Graduate, Masters…
$ major_discipline       <fct> STEM, STEM, STEM, Business Degree, STEM, STEM, …
$ experience             <fct> 0, >10, 5-10, 0, 0, >10, 5-10, >10, 5-10, >10, …
$ company_size           <chr> "Other", "Small", "Other", "Other", "Small", "O…
$ company_type           <chr> "", "Pvt Ltd", "", "Pvt Ltd", "Funded Startup",…
$ last_new_job           <chr> "1", ">4", "never", "never", "4", "1", "1", ">4…
$ training_hour

## The data frame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at large enterprise companies.

In [63]:
#filter columns to only contain student with 10 or more YOE. 
unique(ds_jobs_clean$company_size)

ds_jobs_clean %>% filter(company_size == "Large" | experience == ">10")

[1] "Other"  "Small"  "Micro"  "Large"  "Medium"

student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
<int>,<chr>,<dbl>,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>,<chr>,<int>,<int>
29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,>10,Small,Pvt Ltd,>4,47,0
21651,city_176,0.764,,Has relevant experience,Part time course,Graduate,STEM,>10,Other,,1,24,1
402,city_46,0.762,Male,Has relevant experience,no_enrollment,Graduate,STEM,>10,Micro,Pvt Ltd,>4,18,1
699,city_103,0.920,,Has relevant experience,no_enrollment,Graduate,STEM,>10,Large,Pvt Ltd,>4,123,0
23853,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,5-10,Large,Pvt Ltd,1,108,0
25619,city_61,0.913,Male,Has relevant experience,no_enrollment,Graduate,STEM,0,Large,Pvt Ltd,3,23,0
6588,city_114,0.926,Male,Has relevant experience,no_enrollment,Graduate,STEM,>10,Small,Pvt Ltd,>4,18,0
5764,city_21,0.624,,Has relevant experience,no_enrollment,Graduate,STEM,<5,Large,Pvt Ltd,2,7,0
2156,city_21,0.624,,Has relevant experience,no_enrollment,Graduate,STEM,5-10,Large,Pvt Ltd,never,23,1
